In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # Demo 5: SQL Server カスタムコンテナイメージの作成と配布
 ## 5-1.Dockerfile編集

In [38]:
%%bash
cd ~/SQL_Server_Container
touch Dockerfile
echo 'FROM mcr.microsoft.com/mssql/server:2019-RC1' >> Dockerfile
echo 'COPY ./DevDB.bak /tmp/DevDB.bak' >> Dockerfile
echo 'CMD ["/opt/mssql/bin/sqlservr"]' >> Dockerfile
cat Dockerfile


FROM mcr.microsoft.com/mssql/server:2019-RC1
COPY ./DevDB.bak /tmp/DevDB.bak
CMD ["/opt/mssql/bin/sqlservr"]


 ## 5-2.カスタムコンテナイメージのビルド(数十秒かかる)

In [39]:
%%bash
cd ~/SQL_Server_Container
sudo docker build -t localhost:5000/mssql/devdb:v1 .
sudo docker history localhost:5000/mssql/devdb:v1
sudo docker image list | grep devdb



Step 1/3 : FROM mcr.microsoft.com/mssql/server:2019-RC1
 ---> c5a295efea97
Step 2/3 : COPY ./DevDB.bak /tmp/DevDB.bak
 ---> b2dcb5cd0b50
Step 3/3 : CMD ["/opt/mssql/bin/sqlservr"]
 ---> Running in aad50174a09b
Removing intermediate container aad50174a09b
 ---> ad100f159e98
Successfully built ad100f159e98
Successfully tagged localhost:5000/mssql/devdb:v1
IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
ad100f159e98        1 second ago        /bin/sh -c #(nop)  CMD ["/opt/mssql/bin/sqls…   0B                  
b2dcb5cd0b50        37 seconds ago      /bin/sh -c #(nop) COPY file:b9d59d1908bdbcdf…   3.26MB              
c5a295efea97        2 months ago        /bin/sh -c #(nop)  CMD ["/opt/mssql/bin/sqls…   0B                  
<missing>           2 months ago        /bin/sh -c #(nop)  ENTRYPOINT ["/opt/mssql/b…   0B                  
<missing>           2 months ago        /bin/sh -c /tmp/install.sh                      121

 ## 5-3.ローカルプライベートレジストリ起動

In [42]:
%%bash
sudo docker run -d -p 5000:5000 --name registry -v ~/SQL_Server_Container/RegistryVolume:/var/lib/registry registry


0c577382b5e7d43c2da5c4ad36e6dfe7defb34c5a4c52225e9c52cb01753121e


 ## 5-4.ローカルプライベートレジストリにプッシュ（事前プッシュ済みなので早い）

In [43]:
%%bash
sudo docker push localhost:5000/mssql/devdb:v1


The push refers to repository [localhost:5000/mssql/devdb]
4cde25556db6: Preparing
291d0eb6856c: Preparing
64a2fdccc647: Preparing
25ed665d515e: Preparing
e4e0ec1b4536: Preparing
2de391e51d73: Preparing
d73dd9e65295: Preparing
686245e78935: Preparing
d7ff1dc646ba: Preparing
644879075e24: Preparing
2de391e51d73: Waiting
d73dd9e65295: Waiting
686245e78935: Waiting
d7ff1dc646ba: Waiting
644879075e24: Waiting
291d0eb6856c: Layer already exists
25ed665d515e: Layer already exists
e4e0ec1b4536: Layer already exists
64a2fdccc647: Layer already exists
2de391e51d73: Layer already exists
644879075e24: Layer already exists
d73dd9e65295: Layer already exists
d7ff1dc646ba: Layer already exists
686245e78935: Layer already exists
4cde25556db6: Pushed
v1: digest: sha256:ffe5d637b244a46cc2b091ad688b44b1baa8c238836653c7a74a9ef0e4d76707 size: 2411


 ## 5-5.ローカルプライベートレジストリ確認

In [44]:
%%bash
wget http://localhost:5000/v2/_catalog -q -O /dev/stdout
wget http://localhost:5000/v2/mssql/devdb/tags/list -q -O /dev/stdout


{"repositories":["mssql/devdb"]}
{"name":"mssql/devdb","tags":["v1"]}


 ## 5-6.ローカルに作成したカスタムコンテナイメージを削除（明示的にプライベートレジストリからPULLするため）

In [45]:
%%bash
sudo docker rmi localhost:5000/mssql/devdb:v1
sudo docker image list


Untagged: localhost:5000/mssql/devdb:v1
Untagged: localhost:5000/mssql/devdb@sha256:ffe5d637b244a46cc2b091ad688b44b1baa8c238836653c7a74a9ef0e4d76707
Deleted: sha256:ad100f159e982691fe7f1803686bfba3e40b32e3ff75388a0eb710442b72329c
Deleted: sha256:b2dcb5cd0b50486d012ce7e8b957e672fc49d733ab934088839dd48669b8e5a2
Deleted: sha256:b4dfeb68c049933d2e2c1a6cc8ab054496bf93bd2e0523b588368de83af209b9
REPOSITORY                       TAG                 IMAGE ID            CREATED             SIZE
localhost:5000/mssql/devdb       <none>              496f00011d41        11 minutes ago      1.5GB
mcr.microsoft.com/mssql/server   2019-RC1            c5a295efea97        2 months ago        1.5GB
mcr.microsoft.com/mssql/server   2019-latest         c5a295efea97        2 months ago        1.5GB
mcr.microsoft.com/mssql/server   2017-CU16           d9b9b96627b7        2 months ago        1.36GB
registry                         latest              f32a97de94e1        7 months ago        25.8MB


 ## 5-7.ローカルプライベートレジストリから Pull & Run

In [46]:
%%bash
sudo docker run -e ACCEPT_EULA=Y -e MSSQL_SA_PASSWORD=$PASSWORD -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql19rc1 -d localhost:5000/mssql/devdb:v1
sudo docker ps -a --filter name=sql19rc1
sudo docker image list | grep devdb


39417adea30902332cbc847dd74a6b677057eb4c75c72d492421b87dd6ecbbeb
CONTAINER ID        IMAGE                           COMMAND                  CREATED             STATUS                  PORTS                     NAMES
39417adea309        localhost:5000/mssql/devdb:v1   "/opt/mssql/bin/perm…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql19rc1
localhost:5000/mssql/devdb       v1                  ad100f159e98        2 minutes ago       1.5GB
localhost:5000/mssql/devdb       <none>              496f00011d41        12 minutes ago      1.5GB
Unable to find image 'localhost:5000/mssql/devdb:v1' locally
v1: Pulling from mssql/devdb
b234f539f7a1: Already exists
55172d420b43: Already exists
5ba5bbeb6b91: Already exists
43ae2841ad7a: Already exists
f6c9c6de4190: Already exists
e39f945bda21: Already exists
6689ce95f395: Already exists
ec004dcfdfb5: Already exists
e44708601d04: Already exists
93193a7c97b2: Pulling fs layer
93193a7c97b2: Verifying Checksum
93193a7c97b2:

 ## 5-8.カスタム（バックアップファイルを配置）を確認

In [47]:
%%bash
sudo docker exec sql19rc1 ls -l /tmp


total 3188
-rw-r----- 1 root root 3264512 Oct 26 04:26 DevDB.bak
